In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from datasets import load_dataset

dataset = load_dataset("conll2003")
print(dataset["train"][0])


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [3]:
from transformers import AutoTokenizer
from datasets import DatasetDict
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], 
        truncation=True, 
        is_split_into_words=True,
        padding="max_length",  # Добавляем padding для единообразия
        max_length=128         # Указываем максимальную длину
    )
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:
            # Специальные токены получают метку -100
            if word_idx is None:
                label_ids.append(-100)
            # Для новых слов используем соответствующую метку
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # Для подслов используем -100
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
            
        labels.append(label_ids)
    
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Применяем функцию к датасету
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    batch_size=32,  # Размер батча для обработки
    remove_columns=dataset["train"].column_names  # Удаляем исходные колонки
)

# Устанавливаем формат для PyTorch
tokenized_datasets.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
label_list = dataset["train"].features["ner_tags"].feature.names
# Загрузка модели
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased", 
    num_labels=len(label_list),
    ignore_mismatched_sizes=True  # Игнорировать несоответствие размеров
)

# Исправленные аргументы обучения
args = TrainingArguments(
    output_dir="./ner_model",
    eval_strategy="epoch",  # Изменено с evaluation_strategy на eval_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="no",
    logging_dir='./logs',  # Добавляем директорию для логов
    report_to="none"       # Отключаем отчеты если не используете WandB
)

# Создаем Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"].select(range(2000)),
    eval_dataset=tokenized_datasets["validation"].select(range(500)),
)

# Запускаем обучение
trainer.train()

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.263046


TrainOutput(global_step=125, training_loss=0.4760972900390625, metrics={'train_runtime': 36.6674, 'train_samples_per_second': 54.544, 'train_steps_per_second': 3.409, 'total_flos': 130656646656000.0, 'train_loss': 0.4760972900390625, 'epoch': 1.0})

In [6]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.26304590702056885, 'eval_runtime': 2.5956, 'eval_samples_per_second': 192.633, 'eval_steps_per_second': 12.329, 'epoch': 1.0}
